In [20]:
import pandas as pd
import pybedtools 

In [76]:
#! read gff file
Gh_gff="/public/home/jqyou/data/genome_Ghir.HAU/Ghirsutum_gene_model.gff3"
Chromsome_length='/data/cotton/zhenpingliu/jqyou/TM1/TM1_chr_length.txt'
Gh_DataFrame=pd.read_csv(Gh_gff,comment="#",index_col=None,sep="\t",header=None)
def getFeature_from_gff3(gffDataFrame:pd.DataFrame,featureName:str):
    '''
    get location of genome features, such gene body、Intergenic、promoter、UTR 、exon and intron
    args:
      - gffFileName: @pd.DataFrame
      - featureName: @str(gene、mRNA)
    return:
        list: @str interval of feature
    '''
    tmp=Gh_DataFrame.loc[gffDataFrame[2]==featureName]
    if featureName in ["exon","intron","three_prime_UTR","five_prime_UTR","mRNA"]:
        tmp[8]=tmp[8].apply(lambda x:x.strip(";").split(";")[-1].strip("Parent="))
    else:
        tmp[8]=tmp[8].apply(lambda x:x.strip(";").split(";")[0].strip("ID="))
    ###! get string of interval    
    OutStr="\n".join(tmp.apply(lambda x:x[0]+"\t"+str(x[3])+"\t"+str(x[4])+"\t"+x[8]+"\t"+x[6]+"\t"+featureName,axis=1).values)
    return pybedtools.BedTool(OutStr,from_string=True)
    

    

In [77]:
###################################
# set the feature 
###################################
geneFeature=getFeature_from_gff3(Gh_DataFrame,'gene')
# mRNAFeature=getFeature_from_gff3(Gh_DataFrame,'mRNA')
# exonFeature=getFeature_from_gff3(Gh_DataFrame,'exon')
# UTR_5=getFeature_from_gff3(Gh_DataFrame,'five_prime_UTR')
# UTR_3=getFeature_from_gff3(Gh_DataFrame,'three_prime_UTR')


In [36]:
######genome length File
!cut -f1,2 /public/home/jqyou/data/genome_Ghir.HAU/Ghirsutum_genome.fasta.fai |sort -k1,1 >/data/cotton/zhenpingliu/jqyou/TM1_chr_length.txt 
!cut -f1,2  /public/home/jqyou/data/genome_Garb.HAU/Lachesis_assembly_changed.fa.fai |sort -k1,1 >/data/cotton/zhenpingliu/jqyou/Garb.HAU_chr_length.txt
!cut -f1,2 /public/home/jqyou/data/genome_Grai.HAU/Lachesis_assembly_changed.fa.fai|sort -k1,1 >/data/cotton/zhenpingliu/jqyou/Grai.HAU_chr_length.txt


In [111]:
geneFeature.flank(g=Chromsome_length,b=2001).head()

Ghir_A01	80321912	80323913	Ghir_A01G013980	+	gene
 Ghir_A01	80324566	80326567	Ghir_A01G013980	+	gene
 Ghir_A01	80320151	80322152	Ghir_A01G013970	+	gene
 Ghir_A01	80323048	80325049	Ghir_A01G013970	+	gene
 Ghir_A01	60751986	60753987	Ghir_A01G013100	+	gene
 Ghir_A01	60754357	60756358	Ghir_A01G013100	+	gene
 Ghir_A01	59978269	59980270	Ghir_A01G013050	+	gene
 Ghir_A01	59983471	59985472	Ghir_A01G013050	+	gene
 Ghir_A01	59138876	59140877	Ghir_A01G013020	+	gene
 Ghir_A01	59148861	59150862	Ghir_A01G013020	+	gene
 

In [78]:
###############################################
# get promoter region
# according gene region extract promoter region
###############################################
promoterInterValStr=[]
for index,flankItem in enumerate(geneFeature.flank(g=Chromsome_length,b=2000)):
    if flankItem[4]=="+" and index %2==0:
        tmp=int(flankItem[2])-1
        if tmp>=int(flankItem[1]):
            # promoter less than 1bp
            continue
        else:
            promoterInterValStr.append("\t".join(flankItem[0:2]+[str(tmp),flankItem[3]])+"*"+flankItem[-2]+";\tPromter")
    elif flankItem[4]=="-" and index %2!=0:
        tmp=int(flankItem[1])+1
        if int(flankItem[2])<=tmp:
            continue
        else:
            promoterInterValStr.append("\t".join([flankItem[0],str(tmp)]+flankItem[2:4])+"*"+flankItem[-2]+";\tPromter")
##promoter Bedtools region        
promoterInterValStr=pybedtools.BedTool("\n".join(promoterInterValStr),from_string=True)
promoterInterValStr.saveas("/data/cotton/zhenpingliu/jqyou/TM1/TM1_Promoter.bed")
promoterInterValStr.head()        

Ghir_A01	60758891	60760890	Ghir_A01G013130*-;	Promter
 Ghir_A01	61799375	61801374	Ghir_A01G013160*-;	Promter
 Ghir_A01	60756538	60758537	Ghir_A01G013120*-;	Promter
 Ghir_A01	60985651	60987650	Ghir_A01G013150*-;	Promter
 Ghir_A01	60755259	60757258	Ghir_A01G013110*-;	Promter
 Ghir_A01	60749682	60751681	Ghir_A01G013090*-;	Promter
 Ghir_A01	56329168	56331167	Ghir_A01G012960*-;	Promter
 Ghir_A01	54193887	54195886	Ghir_A01G012740*-;	Promter
 Ghir_A01	55613657	55615656	Ghir_A01G012880*-;	Promter
 Ghir_A01	55572327	55574326	Ghir_A01G012840*-;	Promter
 

In [72]:
###################################################
# get Intergenic region 
# with tem file to get gene region and promoter region
###################################################
from tempfile import NamedTemporaryFile
gene_promoter=NamedTemporaryFile(mode='w+t',encoding='utf-8')
for line in geneFeature:
    #write gene to tmp file
    gene_promoter.write("\t".join(line[0:3])+"\t"+line[3]+"*"+line[4]+"\t"+line[5]+"\n")
for line in promoterInterValStr:
    # write promoter to tmp file
    gene_promoter.write("\t".join(line)+"\n")

mregeInterval=pybedtools.BedTool(gene_promoter.name)
#merge gene and promoter region
#! bed must be sort 
mregeInterval=mregeInterval.sort().merge()
#!complement with the genome length file
#!genome file name variant TM1
Intergenic=mregeInterval.complement(g=Chromsome_length)
Intergenic.saveas("/data/cotton/zhenpingliu/jqyou/D5/D5_Intergenic.bed")



<BedTool(/data/cotton/zhenpingliu/jqyou/D5/D5_Intergenic.bed)>

In [37]:
geneFeature.head()

Ghir_A01	80323913	80324566	Ghir_A01G013980	+	gene
 Ghir_A01	80322152	80323048	Ghir_A01G013970	+	gene
 Ghir_A01	60753987	60754357	Ghir_A01G013100	+	gene
 Ghir_A01	59980270	59983471	Ghir_A01G013050	+	gene
 Ghir_A01	59140877	59148861	Ghir_A01G013020	+	gene
 Ghir_A01	60294264	60295580	Ghir_A01G013060	+	gene
 Ghir_A01	60764363	60764947	Ghir_A01G013140	+	gene
 Ghir_A01	60758567	60758890	Ghir_A01G013130	-	gene
 Ghir_A01	61795448	61799374	Ghir_A01G013160	-	gene
 Ghir_A01	58486311	58487745	Ghir_A01G013000	+	gene
 

In [73]:
########################################
# get Five 5_UTR region
########################################
mrege5_UTR=UTR_5.sort().merge()
########################################
# get three_UTR
# substract the 5UTR
########################################
merge3_UTR=UTR_3.sort().merge()
merge3_UTR=merge3_UTR.subtract(mrege5_UTR)

***** WARNING: File /tmp/pybedtools.k2hbb0nt.tmp has a record where naming convention (leading zero) is inconsistent with other files:
Chr10	22098	22526

***** WARNING: File /tmp/pybedtools.k2hbb0nt.tmp has a record where naming convention (leading zero) is inconsistent with other files:
Chr10	22098	22526



In [74]:
#########################################
# get exon region
# 1.merge 3UT and 5UTR
# exon subtract 3UTR and 5UTR
#########################################
UTR_region=NamedTemporaryFile(mode='w+t',encoding='utf-8')
for line in mrege5_UTR:
    #write gene to tmp file
    UTR_region.write("\t".join(line)+"\n")
for line in merge3_UTR:
    # write promoter to tmp file
    UTR_region.write("\t".join(line)+"\n")
UTRInterval=pybedtools.BedTool(UTR_region.name) 
UTRInterval=UTRInterval.sort().merge()
#! substract UTR region
# print(exonFeature[25203])
exon_Interval=exonFeature.sort().merge().subtract(UTRInterval)
#############################
# intron interval
# gene  substract exon and UTR
#############################
complement_intron=NamedTemporaryFile(mode='w+t',encoding='utf-8')
for line in UTRInterval:
    #write gene to tmp file
    complement_intron.write("\t".join(line)+"\n")
for line in exon_Interval:
    # write promoter to tmp file
    complement_intron.write("\t".join(line)+"\n")

complement_intron_Interval=pybedtools.BedTool(complement_intron.name) 
complement_intron_Interval=complement_intron_Interval.sort().merge()

intron_Interval=geneFeature.subtract(complement_intron_Interval)
# intron_Interval.saveas("/data/cotton/zhenpingliu/jqyou/TM1_intron.bed")

***** WARNING: File /tmp/pybedtools.8d0o8g7i.tmp has a record where naming convention (leading zero) is inconsistent with other files:
Chr10	15385	16482

***** WARNING: File /tmp/pybedtools.8d0o8g7i.tmp has a record where naming convention (leading zero) is inconsistent with other files:
Chr10	15385	16482

***** WARNING: File /tmp/pybedtools.7j5e96d6.tmp has a record where naming convention (leading zero) is inconsistent with other files:
Chr10	7376	9043

***** WARNING: File /tmp/pybedtools.7j5e96d6.tmp has a record where naming convention (leading zero) is inconsistent with other files:
Chr10	7376	9043



In [75]:
######################################################
#get gene ID and stand bo intersect with gene Feature
# mrege5_UTR
# mrege3_UTR
# exon_Interval
# intron_Interval
#####################################################
# mrege5_UTR.saveas("/data/cotton/zhenpingliu/jqyou/TM1_5UTR.bed")
Interset_interval=mrege5_UTR.intersect(geneFeature,loj=True)
UTR5_dict={}
for Interval in Interset_interval:
    UTR5_dict["\t".join(Interval[0:3])]=UTR5_dict.get("\t".join(Interval[0:3]),'')+"*".join(Interval[6:8])+";"
UTR5_str=[ key+"\t"+value+"\t5UTR" for key,value in UTR5_dict.items()]
##save to txt 
pybedtools.BedTool("\n".join(UTR5_str),from_string=True).saveas("/data/cotton/zhenpingliu/jqyou/D5/D5_5UTR.bed")
##########################################################
#3UTR
Interset_interval=merge3_UTR.intersect(geneFeature,loj=True)
UTR5_dict={}
for Interval in Interset_interval:
    UTR5_dict["\t".join(Interval[0:3])]=UTR5_dict.get("\t".join(Interval[0:3]),'')+"*".join(Interval[6:8])+";"
UTR5_str=[ key+"\t"+value+"\t3UTR" for key,value in UTR5_dict.items()]
##save to txt 
pybedtools.BedTool("\n".join(UTR5_str),from_string=True).saveas("/data/cotton/zhenpingliu/jqyou/D5/D5_3UTR.bed")
##########################################################
#! exon interval
Interset_interval=exon_Interval.intersect(geneFeature,loj=True)
UTR5_dict={}
for Interval in Interset_interval:
    UTR5_dict["\t".join(Interval[0:3])]=UTR5_dict.get("\t".join(Interval[0:3]),'')+"*".join(Interval[6:8])+";"
UTR5_str=[ key+"\t"+value+"\texon" for key,value in UTR5_dict.items()]
##save to txt 
pybedtools.BedTool("\n".join(UTR5_str),from_string=True).saveas("/data/cotton/zhenpingliu/jqyou/D5/D5_exon.bed")
##########################################################
#! intron interval
Interset_interval=intron_Interval.intersect(geneFeature,loj=True)
UTR5_dict={}
for Interval in Interset_interval:
    UTR5_dict["\t".join(Interval[0:3])]=UTR5_dict.get("\t".join(Interval[0:3]),'')+"*".join(Interval[9:11])+";"
UTR5_str=[ key+"\t"+value+"\tintron" for key,value in UTR5_dict.items()]
##save to txt 
pybedtools.BedTool("\n".join(UTR5_str),from_string=True).saveas("/data/cotton/zhenpingliu/jqyou/D5/D5_intron.bed")
# 'Ghir_A01\t6687352\t6687761'


***** WARNING: File /tmp/pybedtools.bnlu4ajw.tmp has inconsistent naming convention for record:
Chr01	38	238	Grai_01G000010	-	gene

***** WARNING: File /tmp/pybedtools.bnlu4ajw.tmp has a record where naming convention (leading zero) is inconsistent with other files:
Chr01	38	238	Grai_01G000010	-	gene

***** WARNING: File /tmp/pybedtools.bnlu4ajw.tmp has a record where naming convention (leading zero) is inconsistent with other files:
Chr01	38	238	Grai_01G000010	-	gene

***** WARNING: File /tmp/pybedtools.bnlu4ajw.tmp has inconsistent naming convention for record:
Chr01	38	238	Grai_01G000010	-	gene

***** WARNING: File /tmp/pybedtools.bnlu4ajw.tmp has a record where naming convention (leading zero) is inconsistent with other files:
Chr01	38	238	Grai_01G000010	-	gene

***** WARNING: File /tmp/pybedtools.bnlu4ajw.tmp has a record where naming convention (leading zero) is inconsistent with other files:
Chr01	38	238	Grai_01G000010	-	gene

***** WARNING: File /tmp/pybedtools.bnlu4ajw.tmp has

<BedTool(/data/cotton/zhenpingliu/jqyou/D5/D5_intron.bed)>

In [17]:
#################################################
# not intersect with each other
# merge all the file
# pandas sort it 
#################################################
# cat TM1_Promoter.bed TM1_exon.bed  TM1_intron.bed TM1_5UTR.bed TM1_3UTR.bed|sort -k1,1 -k2,2n >TM1_gene_info.txt
geneData=pd.read_csv("/data/cotton/zhenpingliu/jqyou/TM1/TM1_gene_info.txt",header=None,index_col=None,sep="\t")


In [19]:
for lineindex in range(1,geneData.shape[0],1):
    #!skip the first line 
    if lineindex%100000==0:
        print(lineindex)
    if geneData.iloc[lineindex,1]==geneData.iloc[lineindex-1,2]:
        # print(geneData.iloc[lineindex-1][2])
        geneData.iloc[lineindex-1,2]=geneData.iloc[lineindex-1,2]-1
geneData.to_csv("/data/cotton/zhenpingliu/jqyou/D5/D5_gene_info.txt",header=False,index=False,sep="\t")
 


,0,1,2,3,4
0,Ghir_A01,13137,13159,Ghir_A01G000010*-;,3UTR
1,Ghir_A01,13160,13843,Ghir_A01G000010*-;,intron
2,Ghir_A01,13844,13871,Ghir_A01G000010*-;,3UTR
3,Ghir_A01,13872,14083,Ghir_A01G000010*-;,exon
4,Ghir_A01,14084,14167,Ghir_A01G000010*-;,intron
5,Ghir_A01,14168,14258,Ghir_A01G000010*-;,exon
6,Ghir_A01,14259,15599,Ghir_A01G000010*-;,5UTR
7,Ghir_A01,15600,17599,Ghir_A01G000010*-,Promter
8,Ghir_A01,21156,23155,Ghir_A01G000020*+,Promter
9,Ghir_A01,23156,23362,Ghir_A01G000020*+;,exon
